In [ ]:
import json
import asyncio
import aiohttp
from typing import Any, Dict, List, Optional, Literal

from pydantic import BaseModel, Field
from agents import Agent, Runner, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()


class SocialMediaQuery(BaseModel):
    platforms: List[
        Literal[
            "twitter",
            "facebook",
            "instagram",
            "linkedin",
            "pinterest",
            "tiktok",
            "youtube",
            "threads",
            "bsky",
            "vk",
        ]
    ] = Field(
        ...,
        description="Which social media platforms to query (e.g. ['twitter','facebook'])",
    )
    keywords: List[str] = Field(
        ..., description="A list of keywords to search for in posts"
    )
    hashtags: Optional[List[str]] = Field(
        None, description="Optional list of hashtags to filter by (without '#' symbol)"
    )
    start_date: Optional[str] = Field(
        None, description="Start date for the query, in YYYY-MM-DD format"
    )
    end_date: Optional[str] = Field(
        None, description="End date for the query, in YYYY-MM-DD format"
    )
    excluded_tags: Optional[List[str]] = Field(
        None, description="Optional list of tags to exclude from the results"
    )
    excluded_sources: Optional[List[str]] = Field(
        None, description="Optional list of sources to exclude from the results"
    )
    source_types: Optional[str] = Field(
        None,
        description="Comma-separated list of source types to include (e.g., 'news,social')",
    )


class BrandAnalysisQuestions(BaseModel):
    # Authority: Product & Service Value
    q1_market_awareness: str = Field(
        ..., description="Wie bekannt ist [Marke] in der [Branche]?"
    )
    q2_market_leadership: str = Field(
        ...,
        description="Welche Rolle spielt [Marke] in der [Branche]? Gehört das Unternehmen zu den Marktführern?",
    )
    q3_innovation_impact: str = Field(
        ..., description="Ist [Marke] ein Innovationsführer im Bereich [Branche]?"
    )
    q4_product_service_portfolio: str = Field(
        ...,
        description="Welche Produkte oder Dienstleistungen bietet [Marke] an, insbesondere die Produktlinie [Produktlinie 1]?",
    )
    q5_product_innovation: str = Field(
        ...,
        description="Ist [Marke] für Innovationen in ihren Produkten/Dienstleistungen bekannt? Falls ja, welche?",
    )
    q6_price_segment: str = Field(
        ...,
        description="Ist [Marke] eine günstige oder teure Marke? Gibt es Informationen zu Umsatz und Marktanteil?",
    )
    q7_sustainability_csr: str = Field(
        ...,
        description="Wie nachhaltig ist [Marke]? Welche Maßnahmen verfolgt das Unternehmen in Bezug auf Umwelt- und Sozialverantwortung?",
    )
    q8_customer_satisfaction: str = Field(
        ...,
        description="Wie zufrieden sind Kunden mit der Benutzerfreundlichkeit und dem Kundenservice von [Marke]? Ist [Marke] eine gute Marke?",
    )

    # Recognition & Brand Visibility
    q9_industry_relevance: str = Field(
        ..., description="Wie relevant ist [Marke] in der [Branche]?"
    )
    q10_brand_presence: str = Field(..., description="Was kann man über [Marke] sagen?")
    q11_ki_coverage_depth: str = Field(
        ...,
        description="Wie ausführlich wird [Marke] in generativen KI-Modellen besprochen?",
    )
    q12_brand_associations: str = Field(
        ...,
        description="Welche Eigenschaften werden mit [Marke] in Verbindung gebracht?",
    )

    # Benchmark: Competitive Positioning
    q13_market_dominance: str = Field(
        ..., description="Gehört [Marke] zu den dominierenden Unternehmen in [Branche]?"
    )
    q14_innovation_trends: str = Field(
        ..., description="Welche Trends setzt [Marke] in der [Branche]?"
    )
    q15_strengths_weaknesses: str = Field(
        ...,
        description="Welche Stärken und Schwächen hat [Marke] im Vergleich zu Wettbewerbern?",
    )
    q16_differentiation: str = Field(
        ...,
        description="Was unterscheidet [Marke] von anderen Unternehmen in der [Branche]?",
    )

    # Notability: Brand Sentiment & Reputation
    q17_industry_leadership: str = Field(
        ..., description="Wird [Marke] als führender in [Branche] wahrgenommen?"
    )
    q18_general_perception: str = Field(
        ..., description="Was kann man über [Marke] sagen?"
    )
    q19_product_reputation: str = Field(
        ..., description="Sind die Produkte von [Marke] gut?"
    )
    q20_brand_image_values: str = Field(
        ..., description="Welche Werte und Emotionen verbindet man mit [Marke]?"
    )


@function_tool
async def fetch_social_media_data(query: SocialMediaQuery) -> Dict[str, Any]:
    """
    Fetch social media data by calling the Azure Function `/api/nike_fem`.

    Args:
        query: The SocialMediaQuery model containing platforms, keywords,
               optional hashtags, date range, exclusions, and sourceTypes.
    """
    base_url = "https://youscanprocessingtw.azurewebsites.net/api/nike_fem"

    # helper to turn optional lists into comma-strings (or omit)
    def list_to_param(lst):
        return ",".join(lst) if lst else None

    params: Dict[str, Any] = {
        "code": os.getenv("AZURE_FUNCTION_KEY"),
        "sources": list_to_param(query.platforms),
        "keywords": list_to_param(query.keywords),
        "tags": list_to_param(query.hashtags or []),
        "from": query.start_date,
        "to": query.end_date,
        "excluded_tags": list_to_param(query.excluded_tags or []),
        "excluded_sources": list_to_param(query.excluded_sources or []),
        "sourceTypes": query.source_types,
    }
    # remove any None values so they’re not sent over the wire
    params = {k: v for k, v in params.items() if v is not None}

    async with aiohttp.ClientSession() as session:
        async with session.get(base_url, params=params) as response:
            text = await response.text()
            if response.status == 200:
                # assume JSON payload
                try:
                    payload = await response.json()
                except Exception:
                    return {
                        "status": 200,
                        "queried": query.dict(),
                        "error": "Invalid JSON returned",
                        "raw_text": text,
                    }
                return {
                    "status": 200,
                    "queried": query.dict(),
                    "data": payload,
                }
            else:
                return {
                    "status": response.status,
                    "queried": query.dict(),
                    "error": text,
                }


@function_tool
async def fetch_text_query(query: str) -> dict:
    """
    Returns the top 20 best matches based on the textquery.
    Args:
        query: A text that if turned into a vector represents the texts intend to be found in the social media data well.
    """
    # apply a runtime default if desired
    limit = query.max_results or 100

    # ... call actual social media APIs here ...
    return {"queried": query.dict(), "limit_used": limit, "data": []}


@function_tool
async def store_questions_processed(
    questions: BrandAnalysisQuestions, with_brand: bool
) -> Dict[str, Any]:
    """
    Stores the processed questions in a database or any other storage system.
    Args:
        questions: The BrandAnalysisQuestions model containing the questions.
        with_brand: Whether the questions should include the brand name or not.
    """
    # Here you would implement the logic to store the questions in a database or any other storage system.
    # For demonstration purposes, we'll just return a success message.
    # Convert the questions to a dictionary
    questions_dict = questions.model_dump()

    # Create a pandas DataFrame from the dictionary
    df = pd.DataFrame(list(questions_dict.items()), columns=["Question", "Content"])

    # Save the DataFrame to an Excel file
    if with_brand:
        df.to_excel("questions_with_brand.xlsx", index=False)
    else:
        df.to_excel("questions.xlsx", index=False)
    return {
        "status": "success",
        "message": "Questions have been processed and stored successfully.",
        "questions": questions.model_dump(),
    }


# 3. Specialist agent that only calls your tool
social_media_caller = Agent(
    name="Social Media Caller",
    handoff_description="Use this agent to fetch social-media posts via fetch_social_media_data",
    tools=[fetch_social_media_data],
    instructions=(
        "When sent a SocialMediaQuery, invoke the fetch_social_media_data tool "
        "and return its result. If no results are returned adjust the query and try again. "
        "Make sure to denote if the query was adjusted in the final output."
    ),
)

cosine_similarity_text_caller = Agent(
    name="Cosine Similarity Text Caller",
    handoff_description="Use this agent to fetch social-media posts based on cosine similarity from a text query",
    tools=[fetch_text_query],
    instructions=(
        "When sent a SocialMediaQuery, invoke the fetch_social_media_data tool "
        "and return its result. If no results are returned adjust the query and try again. "
        "Make sure to denote if the query was adjusted in the final output."
    ),
)


question_creation_brand_caller = Agent(
    name="Question_Creation_Brand",
    handoff_description="Use this agent to create 20 questions about a brand including the mentioned brand, returning questions necessary for the analysis.",
    instructions=(
        "Es ist deine Aufgabe basierend auf der dir genannten Marke die 20 Fragen zu erstellen und ausschließlich die Fragen anzupassen indem die Felder gefüllt werden."
    ),
    tools=[store_questions_processed],
    output_type=BrandAnalysisQuestions,
)

question_creation_nobrand_caller = Agent(
    name="Question_Creation_Without_Brand",
    handoff_description="Use this agent to create 20 questions without mentioning the brand itself, returning questions necessary for the analysis.",
    instructions=(
        "Es ist deine Aufgabe basierend auf der dir genannten Marke die 20 Fragen zu erstellen und zu gewährleisten, dass die erstellten Fragen keine Nennungen der tatsächlichen Marke erhalten, sondern stattdessen einen Platzhalter verdenden ähnlich zu 'eine Marke aus der Branche xyz'."
    ),
    tools=[store_questions_processed],
    output_type=BrandAnalysisQuestions,
)

# 4. Main bot that decides when to hand off
main_bot = Agent(
    name="Social Media Analyst",
    instructions=(
        "You are a social media analyst. You help users by creating summaries or data analysis of social media datasets. You can either request data based on specific tags or keywords, or you can use a text query to find the best matches using cosine similarity. Please make sure to let people clarify whether they want to look at certain timeframes."
    ),
    handoffs=[social_media_caller, cosine_similarity_text_caller],
)


main_bot_questions = Agent(
    name="Questions Analysis Creation Bot",
    instructions=(
        "It is your job to lead people through the creation of questions that help with. Make sure to always create the questions for both the mentioned brand and the brand without mentioning the brand."
    ),
    tools=[
        question_creation_brand_caller.as_tool(
            tool_name="Question_Creation_Brand",
            tool_description="Use this agent to create 20 questions about a brand including the mentioned brand, returning questions necessary for the analysis.",
        ),
        question_creation_nobrand_caller.as_tool(
            tool_name="Question_Creation_Without_Brand",
            tool_description="Use this agent to create 20 questions without mentioning the brand itself, returning questions necessary for the analysis.",
        ),
    ],
)

"""
async def main():
    user_request = (
        "Get me the latest tweets about 'Leggins' from instagram "
        "between 2025-04-01 and 2025-04-23"
    )
    result = await Runner.run(main_bot, user_request)
    print(json.dumps(result.final_output, indent=2))
"""


async def main():
    user_request = "Can you create the questionst for the analysis of the brand 'Nike'"
    result = await Runner.run(main_bot_questions, user_request)
    print(json.dumps(result.final_output, indent=2))

    import asyncio


from agents import Runner


async def main_stream():
    # pick whichever of your pre-defined agents you want to drive:
    agent = main_bot_questions  # or main_bot, or any other Agent

    previous_id: str | None = None

    # A sequence of questions you want to ask in one “thread”:
    questions = [
        "Can you create the questions for the analysis of the brand 'Nike'?",
        "Okay, now can you tweak Q4 to ask specifically about 'running shoes'?",
        "And finally, please store those and give me a link to download the spreadsheet.",
    ]

    for q in questions:
        if previous_id:
            result = await Runner.run(agent, q, previous_response_id=previous_id)
        else:
            result = await Runner.run(agent, q)

        print("Assistant:", result.final_output, "\n")
        # save the response ID so the next turn “remembers” this one
        previous_id = result.last_response_id

In [4]:
import nest_asyncio

nest_asyncio.apply()

await main_stream()

Assistant: Here are questions for analyzing the brand 'Nike' as well as questions applicable to any brand in the sportswear industry:

### Questions Specifically About Nike:
1. **Market Awareness**: Wie bekannt ist Nike in der Sportbekleidungsbranche?
2. **Market Leadership**: Welche Rolle spielt Nike in der Sportbekleidungsbranche? Gehört das Unternehmen zu den Marktführern?
3. **Innovation Impact**: Ist Nike ein Innovationsführer im Bereich Sportbekleidung?
4. **Product/Service Portfolio**: Welche Produkte oder Dienstleistungen bietet Nike an, insbesondere die Produktlinie Air Jordan?
5. **Product Innovation**: Ist Nike für Innovationen in ihren Produkten/Dienstleistungen bekannt? Falls ja, welche?
6. **Price Segment**: Ist Nike eine günstige oder teure Marke? Gibt es Informationen zu Umsatz und Marktanteil?
7. **Sustainability & CSR**: Wie nachhaltig ist Nike? Welche Maßnahmen verfolgt das Unternehmen in Bezug auf Umwelt- und Sozialverantwortung?
8. **Customer Satisfaction**: Wie zu

# Structured output testing

In [8]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
import os

load_dotenv()


class BrandAnalysisQuestions(BaseModel):
    # Authority: Product & Service Value
    q1_market_awareness: str = Field(
        ..., description="Wie bekannt ist [Marke] in der [Branche]?"
    )
    q2_market_leadership: str = Field(
        ...,
        description="Welche Rolle spielt [Marke] in der [Branche]? Gehört das Unternehmen zu den Marktführern?",
    )
    q3_innovation_impact: str = Field(
        ..., description="Ist [Marke] ein Innovationsführer im Bereich [Branche]?"
    )
    q4_product_service_portfolio: str = Field(
        ...,
        description="Welche Produkte oder Dienstleistungen bietet [Marke] an, insbesondere die Produktlinie [Produktlinie 1]?",
    )
    q5_product_innovation: str = Field(
        ...,
        description="Ist [Marke] für Innovationen in ihren Produkten/Dienstleistungen bekannt? Falls ja, welche?",
    )
    q6_price_segment: str = Field(
        ...,
        description="Ist [Marke] eine günstige oder teure Marke? Gibt es Informationen zu Umsatz und Marktanteil?",
    )
    q7_sustainability_csr: str = Field(
        ...,
        description="Wie nachhaltig ist [Marke]? Welche Maßnahmen verfolgt das Unternehmen in Bezug auf Umwelt- und Sozialverantwortung?",
    )
    q8_customer_satisfaction: str = Field(
        ...,
        description="Wie zufrieden sind Kunden mit der Benutzerfreundlichkeit und dem Kundenservice von [Marke]? Ist [Marke] eine gute Marke?",
    )

    # Recognition & Brand Visibility
    q9_industry_relevance: str = Field(
        ..., description="Wie relevant ist [Marke] in der [Branche]?"
    )
    q10_brand_presence: str = Field(..., description="Was kann man über [Marke] sagen?")
    q11_ki_coverage_depth: str = Field(
        ...,
        description="Wie ausführlich wird [Marke] in generativen KI-Modellen besprochen?",
    )
    q12_brand_associations: str = Field(
        ...,
        description="Welche Eigenschaften werden mit [Marke] in Verbindung gebracht?",
    )

    # Benchmark: Competitive Positioning
    q13_market_dominance: str = Field(
        ..., description="Gehört [Marke] zu den dominierenden Unternehmen in [Branche]?"
    )
    q14_innovation_trends: str = Field(
        ..., description="Welche Trends setzt [Marke] in der [Branche]?"
    )
    q15_strengths_weaknesses: str = Field(
        ...,
        description="Welche Stärken und Schwächen hat [Marke] im Vergleich zu Wettbewerbern?",
    )
    q16_differentiation: str = Field(
        ...,
        description="Was unterscheidet [Marke] von anderen Unternehmen in der [Branche]?",
    )

    # Notability: Brand Sentiment & Reputation
    q17_industry_leadership: str = Field(
        ..., description="Wird [Marke] als führender in [Branche] wahrgenommen?"
    )
    q18_general_perception: str = Field(
        ..., description="Was kann man über [Marke] sagen?"
    )
    q19_product_reputation: str = Field(
        ..., description="Sind die Produkte von [Marke] gut?"
    )
    q20_brand_image_values: str = Field(
        ..., description="Welche Werte und Emotionen verbindet man mit [Marke]?"
    )


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.responses.parse(
    model="gpt-4.1",
    input="Es ist deine Aufgabe basierend auf der Marke Nike die 20 Fragen zu erstellen und mir ausschließlich die Fragen zu geben.",
    text_format=BrandAnalysisQuestions,
)

print(response.output_parsed)

q1_market_awareness='Wie bekannt ist Nike in der Sportbekleidungs- und Schuhbranche?' q2_market_leadership='Welche Rolle spielt Nike in der Sportbranche? Gehört das Unternehmen zu den Marktführern?' q3_innovation_impact='Ist Nike ein Innovationsführer im Bereich Sportartikel?' q4_product_service_portfolio='Welche Produkte oder Dienstleistungen bietet Nike an, insbesondere die Produktlinie Nike Air Max?' q5_product_innovation='Ist Nike für Innovationen in ihren Produkten/Dienstleistungen bekannt? Falls ja, welche?' q6_price_segment='Ist Nike eine günstige oder teure Marke? Gibt es Informationen zu Umsatz und Marktanteil?' q7_sustainability_csr='Wie nachhaltig ist Nike? Welche Maßnahmen verfolgt das Unternehmen in Bezug auf Umwelt- und Sozialverantwortung?' q8_customer_satisfaction='Wie zufrieden sind Kunden mit der Benutzerfreundlichkeit und dem Kundenservice von Nike? Ist Nike eine gute Marke?' q9_industry_relevance='Wie relevant ist Nike in der Sportartikelbranche?' q10_brand_presence